<a href="https://colab.research.google.com/github/MinseoYoo/bitamin_nlp_2024_2/blob/Minseo/%EC%B5%9C%EC%A2%85%EC%BD%94%EB%93%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
!pip install u sentence_transformers

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# kogpt api 확인을 위한 허깅페이스 로그인
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineGrained).
The token `token2` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate w

In [ ]:
import re
import pandas as pd
import torch
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel, AutoTokenizer, AutoModelForCausalLM, AutoModelForSeq2SeqLM, Trainer, TrainingArguments
from datasets import load_dataset, Dataset
from sentence_transformers import SentenceTransformer, util

In [ ]:
device = 'cuda'

In [ ]:
# 해당 함수 사용시 대화뭉치 앞에 summarize: 를 붙여주기
# 요약 코드
def summarize(dialogue, device=device):
    tokenizer = AutoTokenizer.from_pretrained("Xcz2568/T5-summarization-Korean1")
    model = AutoModelForSeq2SeqLM.from_pretrained("Xcz2568/T5-summarization-Korean1").to(device)
    input_ids = tokenizer(dialogue, return_tensors="pt", padding=True,
                          truncation=True, max_length=1048).input_ids.to(device)
    output = model.generate(input_ids, max_length=100, num_beams=5, early_stopping=True)
    summary = tokenizer.decode(output[0], skip_special_tokens=True)
    return summary

In [ ]:
# 다음 주제 생성을 위한 kogpt 모델
tokenizer = AutoTokenizer.from_pretrained(
  'kakaobrain/kogpt', revision='KoGPT6B-ryan1.5b-float16',
  bos_token='[BOS]', eos_token='[EOS]', unk_token='[UNK]', pad_token='[PAD]', mask_token='[MASK]'
)

model = AutoModelForCausalLM.from_pretrained(
  'kakaobrain/kogpt', revision='KoGPT6B-ryan1.5b-float16',
  pad_token_id=tokenizer.eos_token_id,
  torch_dtype='auto', low_cpu_mem_usage=True
).to(device='cuda', non_blocking=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.51M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/839 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/12.3G [00:00<?, ?B/s]

Some weights of the model checkpoint at kakaobrain/kogpt were not used when initializing GPTJForCausalLM: ['transformer.h.0.attn.bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.1.attn.bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.10.attn.bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.11.attn.bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.12.attn.bias', 'transformer.h.12.attn.masked_bias', 'transformer.h.13.attn.bias', 'transformer.h.13.attn.masked_bias', 'transformer.h.14.attn.bias', 'transformer.h.14.attn.masked_bias', 'transformer.h.15.attn.bias', 'transformer.h.15.attn.masked_bias', 'transformer.h.16.attn.bias', 'transformer.h.16.attn.masked_bias', 'transformer.h.17.attn.bias', 'transformer.h.17.attn.masked_bias', 'transformer.h.18.attn.bias', 'transformer.h.18.attn.masked_bias', 'transformer.h.19.attn.bias', 'transformer.h.19.attn.masked_bias', 'transformer.h.2.attn.bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.20.attn.bias'

In [ ]:
# 요약 코드를 돌려 만든 csv로 성능 평가
df_new = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/비타민 14기/학기 NLP/summary_output_valid.csv')

In [ ]:
df_new

,processed_dialogue,reference,summary
0,<extra_id_0> 아직 엄마를 못하니까 그러지 <extra_id_1> 엄마해 ...,기분 좋을 때는 아빠 급하거나 울 때는 엄마라고 한다.,기분이 좋을 때는 아빠를 급하거나 울 때는 엄마를 한다.
1,<extra_id_0> 오늘 바로 딥갔어? 아님 놀았니 <extra_id_1> 집갔...,오늘 바로 집에 갔지만 잘 쉬지는 못했다.,오늘 집에 가서 멍을 때렸는데 휴식이 되지 못했다.
2,<extra_id_0> ㅋㅋㅋ남동생이랑 이야기해서 싸운거야 여자친구 생겼어 #@이름...,프로필 사진(프사)을 보니 여자친구가 생긴 것 같고 동생이 싫다고 이야기한다.,동생이 남자친구(여친)가 생겼는데 핸드폰 번호가 옛날 것이라 안 지웠는데 여자친구(...
3,<extra_id_0> 나 잘생긴 사람 만나고 싶다 어디가야 잘생긴 놈 있나 알려주...,충재 씨가 요즘 마음에 들어 잘 생긴 남자가 만나고 싶고 까탈스럽게 좋은 사람을 만...,잘생긴 사람을 만나고 싶다고 하자 너무 초식남 같으면 별로라고 한다.
4,<extra_id_0> 내뒤에어떤할아버지가 계속서서 이상한 소리내 진짜무서워 <ex...,뒤에 있는 할아버지가 이상한 소리를 내서 무섭다고 했다.,뒤에 어떤 할아버지가 계속 이상한 소리를 내서 무서웠다.
...,...,...,...
34999,<extra_id_0> 경복궁 갓다 나와서 밥 먹고 <extra_id_1> #@이모...,경복궁 갔다가 닭볶음탕을 먹으려고 하는데 무명 식당은 정갈한 느낌은 아니다.,경복궁에서 밥을 먹고 무명식당에서 닭볶음탕을 먹었다.
35000,<extra_id_0> 1일차에는 11시쯤 도착하니까 제주도에 저거 이용못하고 2일...,우도 빼고 다른 곳에 가도 된다고 해서 우도는 포기 못 한다고 한다.,제주도에 도착하면 우도를 못 가서 버스 투어를 이용할 수 없다.
35001,<extra_id_0> 가고싶은곳 찾아 알차게 3일 놀아야하니까 아주 꽉꽉 야아무지...,3일 동안 가고 싶은 곳의 사진을 보내고 있다.,엄마랑 이모가 갈 곳을 대충 보내보라고 해서 그냥 가고 싶은 곳을 가면 된다고 한다.
35002,<extra_id_0> 근데 호텔이랑 게스트하우스랑 아니 게스트하우스말고 호텔하는게...,게스트 하우스는 치안이 걱정되는데 피렌체는 치안이 괜찮고 로마가 안 좋다.,호텔보다 게스트하우스가 치안이 더 좋지만 피렌체는 치안이 괜찮다.


In [ ]:
# 추출한 요약으로 생성 및 최적문장 생성
def generate_answer(prompt, device=device):
    _ = model.eval()
    with torch.no_grad():
        tokens = tokenizer.encode(prompt, return_tensors='pt').to(device)
        gen_tokens = model.generate(
            tokens,
            do_sample=True,
            temperature=0.8,
            max_length=200,
            eos_token_id=tokenizer.eos_token_id,
            top_p=0.9,
            top_k=50,
            repetition_penalty=1.2
        )
        generated = tokenizer.batch_decode(gen_tokens)[0]

    # 문장을 분리하여 리스트화
    sentences = [sentence.strip() for sentence in re.split(r'(?<=[.!?])\s*', generated) if sentence.strip()]
    return sentences

# 문장과 질문의 유사도를 계산
def calculate_question(dialogue, stlist):
    stlist = stlist[1:]
    st_model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS').to(device)
    st_dia_embedding = st_model.encode(dialogue)
    st_que_embedding = st_model.encode(stlist)

    similarities = util.cos_sim(st_dia_embedding, st_que_embedding)
    best_question_index = similarities.argmax()
    second_question_index = similarities.argsort()[0][-2]
    return stlist[best_question_index], stlist[second_question_index]

In [ ]:
# df_new에서 평가 수행
summary_train = df_new['summary'][3]
dialogue_train = df_new['processed_dialogue'][3]
prompt_train = 'Q:' + summary_train + 'A:'
st_train = generate_answer(prompt_train)
bq, sq = calculate_question(dialogue_train, st_train)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.02k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/394 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/336k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/967k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
print(bq)
print(sq)

그건 그렇지 않다.
자신이 없어 보이면 다른 여자들 눈에도 그렇게 보기 마련이다.


## 실제 대화로 성능평가

In [ ]:
text ='''summarize:
"야, 이번 방학에 여행 가기로 한 거 기억하지?"
"그렇지! 근데 어디로 갈지 아직 못 정했잖아 ㅋㅋ"
"그러니까. 국내로 갈지 해외로 갈지도 못 정했네…"
"솔직히 해외 가고 싶긴 한데, 예산이 좀 빡셀 듯."
"그럼 국내로 가자. 강원도? 제주도? 어디가 좋아?"
"음… 강원도는 맨날 갔으니까 이번엔 제주도 어때?"
"오, 제주 좋다! 바다도 보고 맛집도 많잖아."
"ㅇㅇ! 나 흑돼지 진짜 먹고 싶었어 ㅋㅋ"
"그럼 숙소랑 비행기표는 내가 알아볼게. 너는 맛집 리스트 좀 찾아줘."
"오케이! 바로 서칭 들어간다~"
"참, 몇 박 며칠로 갈 건지도 정해야겠다."
"3박 4일 정도가 딱 좋지 않을까? 휴가도 그 정도 낼 수 있을 듯."
"좋아, 일정 대충 짜서 단톡방에 올릴게!"
"굿굿! 여행 너무 기대된다~"
'''

dialogue_test = text
summary_test = summarize(dialogue_test, device=device)
prompt_test = 'Q:' + summary_test + 'A:'
st_test = generate_answer(prompt_test, device=device)
bq1, sq1 = calculate_question(dialogue_test, st_test)

print("대화요약: ", summary_test)
print("다음 대화 주제 추천 1위: ", bq1)
print("다음 대화 주제 추천 2위: ", sq1)

대화요약:  이번 방학에 강원도와 제주도로 여행을 가기로 했다.
다음 대화 주제 추천 1위:  A:가족이나 친구들끼리 떠날 경우엔 아무래도 안전이 보장된 해외보다는 내가 계획해서 떠나기 좋은 국내여행이 더 매력적이지 않을까 싶다.
다음 대화 주제 추천 2위:  1순위, 볼거리가 풍부한 곳2순위, 자연경관이 좋은 곳3순위, 사람이 많이 몰리고 유명한 관광지4순위, 사람이 적어 한적하고 조용한 곳 5순위, 날씨나 계절 상관없이 항상 갈 수 있는 여행지 Q:최근 해외여행과 국내여행 중 어디로 가볼 것인가 고민이다.
